In [3]:
import pygame 
import chess
import pandas as pd
import sys

# CONFIGURAZIONE 
LARGHEZZA = 640
DIM_CASELLA = LARGHEZZA // 8
FPS = 30
COLORI = [(240, 217, 181), (181, 136, 99)]  # chiaro/scuro

# CARICA I DATI DAL FILE EXCEL 
df = pd.read_excel("Scacchi_con_id_campione.xlsx")

# === INPUT: scegli la partita ===
id_input = input("Inserisci l'ID della partita (es. aa0001): ").strip()
partita = df[df["game_id"] == id_input]

if partita.empty:
    print("❌ ID partita non trovato.")
    sys.exit()

mosse_str = partita.iloc[0]["Moves"]
mosse = mosse_str.strip().split()
giocatore_bianco = partita.iloc[0]["White"]
giocatore_nero = partita.iloc[0]["Black"]

# CREA TUTTI GLI STATI DELLA PARTITA
board = chess.Board()
stati = [board.copy()]
for mossa in mosse:
    try:
        board.push_san(mossa)
        stati.append(board.copy())
    except:
        break

# INIZIALIZZA PYGAME
pygame.init()
screen = pygame.display.set_mode((LARGHEZZA, LARGHEZZA + 80))  # +40 nomi +40 bottoni
pygame.display.set_caption(f"{giocatore_bianco} vs {giocatore_nero}")
clock = pygame.time.Clock()
font = pygame.font.SysFont("Arial", 20)
indice = 0

# CARICAMENTO IMMAGINI DEI PEZZI
pezzi = {}
mappa_simboli = {
    "P": "Pbianco", "N": "Nbianco", "B": "Bbianco", "R": "Rbianco", "Q": "Qbianco", "K": "Kbianco",
    "p": "pnero", "n": "nnero", "b": "bnero", "r": "rnero", "q": "qnero", "k": "knero"
}
for simbolo, file_img in mappa_simboli.items():
    img = pygame.image.load(f"pezzi/{file_img}.png")
    pezzi[simbolo] = pygame.transform.scale(img, (DIM_CASELLA, DIM_CASELLA))

# BOTTONI GRAFICI 
AUTO_PLAY = False
auto_timer = 0
INTERVALLO = 800  # ms tra mosse

bottoni = {
    "start": pygame.Rect(20, LARGHEZZA + 45, 30, 30),
    "prev":  pygame.Rect(75, LARGHEZZA + 45, 30, 30),
    "auto":  pygame.Rect(120, LARGHEZZA + 45, 60, 30),
    "next":  pygame.Rect(200, LARGHEZZA + 45, 30, 30),
    "end":   pygame.Rect(260, LARGHEZZA + 45, 30, 30),
}

# FUNZIONE DI DISEGNO
def disegna(posizione):
    # Scacchiera
    for r in range(8):
        for c in range(8):
            colore = COLORI[(r + c) % 2]
            pygame.draw.rect(screen, colore, (c*DIM_CASELLA, r*DIM_CASELLA, DIM_CASELLA, DIM_CASELLA))

    # Pezzi
    for square in chess.SQUARES:
        pezzo = posizione.piece_at(square)
        if pezzo:
            col = square % 8
            riga = 7 - (square // 8)
            simbolo = pezzo.symbol()
            img = pezzi.get(simbolo)
            if img:
                screen.blit(img, (col*DIM_CASELLA, riga*DIM_CASELLA))

    # Barra nomi
    pygame.draw.rect(screen, (30, 30, 30), (0, LARGHEZZA, LARGHEZZA, 40))
    testo = font.render(f"{giocatore_bianco} vs {giocatore_nero}", True, (255, 255, 255))
    screen.blit(testo, (LARGHEZZA // 2 - testo.get_width() // 2, LARGHEZZA + 10))

    # Barra pulsanti
    pygame.draw.rect(screen, (200, 200, 200), (0, LARGHEZZA + 40, LARGHEZZA, 40))

    # ⏮
    pygame.draw.polygon(screen, (0, 0, 0), [(30, LARGHEZZA + 60), (45, LARGHEZZA + 50), (45, LARGHEZZA + 70)])
    pygame.draw.line(screen, (0, 0, 0), (25, LARGHEZZA + 50), (25, LARGHEZZA + 70), 3)
    # ◀
    pygame.draw.polygon(screen, (0, 0, 0), [(85, LARGHEZZA + 60), (100, LARGHEZZA + 50), (100, LARGHEZZA + 70)])
    # ▶ / ❚❚
    colore_play = (100, 200, 100) if AUTO_PLAY else (180, 180, 180)
    pygame.draw.rect(screen, colore_play, bottoni["auto"])
    simbolo = "❚❚" if AUTO_PLAY else "▶"
    screen.blit(font.render(simbolo, True, (0, 0, 0)), (135, LARGHEZZA + 46))
    # ▶
    pygame.draw.polygon(screen, (0, 0, 0), [(215, LARGHEZZA + 50), (230, LARGHEZZA + 60), (215, LARGHEZZA + 70)])
    # ⏭
    pygame.draw.polygon(screen, (0, 0, 0), [(275, LARGHEZZA + 50), (290, LARGHEZZA + 60), (275, LARGHEZZA + 70)])
    pygame.draw.line(screen, (0, 0, 0), (295, LARGHEZZA + 50), (295, LARGHEZZA + 70), 3)

# LOOP PRINCIPALE
running = True
while running:
    dt = clock.tick(FPS)
    auto_timer += dt

    for evento in pygame.event.get():
        if evento.type == pygame.QUIT:
            running = False
        elif evento.type == pygame.KEYDOWN:
            if evento.key == pygame.K_RIGHT and indice < len(stati) - 1:
                indice += 1
            elif evento.key == pygame.K_LEFT and indice > 0:
                indice -= 1
        elif evento.type == pygame.MOUSEBUTTONDOWN:
            x, y = evento.pos
            if bottoni["start"].collidepoint(x, y):
                indice = 0
            elif bottoni["prev"].collidepoint(x, y) and indice > 0:
                indice -= 1
            elif bottoni["next"].collidepoint(x, y) and indice < len(stati) - 1:
                indice += 1
            elif bottoni["end"].collidepoint(x, y):
                indice = len(stati) - 1
            elif bottoni["auto"].collidepoint(x, y):
                AUTO_PLAY = not AUTO_PLAY
                auto_timer = 0

    if AUTO_PLAY and auto_timer >= INTERVALLO:
        if indice < len(stati) - 1:
            indice += 1
            auto_timer = 0
        else:
            AUTO_PLAY = False

    disegna(stati[indice])
    pygame.display.flip()

pygame.quit()
sys.exit()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


Inserisci l'ID della partita (es. aa0001):  bf0016


SystemExit: 

C:\Users\ferro\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
